In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

In [2]:
def compute_distance(x,y):
    m = np.empty([len(y),len(x)])
    for i in range(len(y)):
        # m[i] = np.linalg.norm(A-B[i], 1, axis=1)
        m[i] = np.abs( x - y[i] ).sum(axis=1)
    return np.min(m, axis = 0).sum() / len(x)

In [3]:
#corpdir = '/fileserver/nmec-handwriting/stil-writing-corpus-processed/seamfeats/FR'
corpdir = '/fileserver/icdar13/jimbo/processed/seamfeats/'
#corpdir = '/fileserver/icdar13/jimbo/processed/TASfeats/'

files = os.listdir(corpdir)
files.sort()

feature_map = []
for filename in files:
    path = corpdir + "/" + filename
    if os.path.isfile(path):
        feature_file = open(path, 'r').read().splitlines()
        alphabet = np.array([np.fromstring(line, dtype=np.float32, sep=' ')[1:129] for line in feature_file])
        feature_map.append(alphabet)
feature_map = np.array(feature_map)

In [4]:
feature_map[0].shape

(110, 128)

In [5]:
compute_distance(feature_map[0], feature_map[1])

0.77114568081769075

In [6]:
metric = []
for image in feature_map:
    metricline = [np.array([compute_distance(image, other) for other in feature_map])]
    metric += metricline

metric = np.array(metric)
F = -metric
np.fill_diagonal(F, -sys.maxint)

In [7]:
soft_correct = 0
hard_correct = 0
total_num = 0

k = 10
g = 4
max_top = g - 1

for j, i in enumerate(F):
    topk = i.argsort()[-k:]
    if j/g in topk/g:
        soft_correct += 1
    total_num += 1
    hardsample = sum([1 for jj in (j/g == topk[-max_top:]/g) if jj])
    if hardsample == max_top:
        hard_correct += 1
        print j

print "Top %d = %f" %(k, (soft_correct + 0.0) / total_num)
print "Top %d = %f" %(max_top, (hard_correct + 0.0) / total_num)

33
48
49
50
106
154
172
189
232
234
248
249
250
251
258
263
277
366
Top 10 = 0.992500
Top 3 = 0.045000
